<a href="https://colab.research.google.com/github/kgautam2103/Zoom_speech2text_AWS_public/blob/main/zoom_speech2text_utility_functions_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [ ]:

def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [ ]:
def double_encode_uuid(meeting_uuid):
  encoded_uuid = urlencode({'id':meeting_uuid})
  double_encoded_uuid = urlencode({'id':encoded_uuid[3:]})
  return double_encoded_uuid[3:]

In [ ]:
def get_recording_download_url(response_json):
  for x in response_json['recording_files']:
    if x['file_extension']=='M4A':
      return x['download_url'],x['recording_start']

In [ ]:
def m4a_to_wav(input_filepath):
  (path, file_extension) = os.path.splitext(input_filepath)
  file_extension_final = file_extension.replace('.', '')
  track = AudioSegment.from_file(input_filepath,
                        file_extension_final)
  output_filepath = input_filepath.replace('m4a','wav')
  file_handle=track.export(output_filepath, format='wav')
  return output_filepath



In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [ ]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [ ]:
def s3_download_file(s3_bucket_name,file_name):
  s3 = boto3.resource('s3')

  try:
      s3.Bucket(s3_bucket_name).download_file(KEY, file_name)
  except botocore.exceptions.ClientError as e:
      if e.response['Error']['Code'] == "404":
          print("The object does not exist.")
      else:
          raise

In [ ]:
def s3_delete_file(s3_bucket_name,file_name):
  s3 = boto3.resource('s3')
  try:
      s3_obj = s3.Object(s3_bucket_name, file_name)
      s3_obj.delete()
  except botocore.exceptions.ClientError as e:
      if e.response['Error']['Code'] == "404":
          print("The object does not exist.")
      else:
          raise

In [ ]:
def s3_upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
def aws_transcribe(job_name,job_uri,aws_region):
  transcribe = boto3.client('transcribe',region_name=aws_region)
  transcribe.start_transcription_job(
      TranscriptionJobName=job_name,
      Media={'MediaFileUri': job_uri},
      MediaFormat='wav',
      LanguageCode='en-US',
      Settings={'ShowSpeakerLabels':True,'MaxSpeakerLabels':4}
  )
  while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    #print("Not ready yet...")
    time.sleep(5)
  return status

In [ ]:

def google_transcribe(cwd,bucket_name,filename):
  gcs_uri = 'gs://'+bucket_name+'/'+filename
  local_filepath=cwd+'/'+filename
  client = speech.SpeechClient()
  frame_rate, channels=frame_rate_channel(local_filepath)
  print(frame_rate)
  print(channels)
  audio = {'uri':gcs_uri}
  config = {"encoding":"LINEAR16","sample_rate_hertz":frame_rate,"language_code":'en-US',"alternative_language_codes":['en-IN','hi-IN'],
            "enable_automatic_punctuation":True,"enable_speaker_diarization":True}

  # Detects speech in the audio file
  operation = client.long_running_recognize(audio=audio,config=config)
  response = operation.result()
  transcript=''

  for result in response.results:
      transcript += result.alternatives[0].transcript

  delete_blob(bucket_name, filename)
  return transcript

In [ ]:
def write_transcripts(transcript_filename,transcript):
    f= open(transcript_filename,"w+")
    f.write(transcript)
    f.close()